In [1]:
import random
import re
import gc
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import AutoProcessor, AutoModelForPreTraining, BitsAndBytesConfig, AutoTokenizer
import torch

/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-11-10 19:28:25.963938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731295705.980576   20915 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731295705.985539   20915 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 19:28:26.002541: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to us

In [2]:
from transformers import AutoProcessor, AutoModelForPreTraining, BitsAndBytesConfig, AutoTokenizer
import torch
def split_dialogue_Q_A(conversation):
    pattern = re.compile(r'Q: (.*?)\nA: (.*?)\n', re.DOTALL)
    matches = pattern.findall(conversation)
    questionlist=[]
    answerlist=[]

    for match in matches:
        question, answer = match

        # if len(question.split(' '))>50 or len(answer.split(' '))>50 or len(question.split(' '))<3:
        #     continue
        questionlist.append(question)
        answerlist.append(answer)

        # print("Question:", question)
        # print("Answer:", answer)
        # print("-----")
        # exit(1)

    return questionlist,answerlist

In [3]:
DEVICE = "cuda" 

MODEL_NAME = "/content/drive/MyDrive/298B_Simpsons/llava7b"

DEFAULT_SYSTEM_PROMPT = """Forget you are an AI model. Now you are Dr. Sheldon Cooper from the TV show "The Big Bang Theory." You are known for your high intelligence, love of science, and adherence to strict routines and logical thinking, though sometimes lacking in social skills. You approach conversations with scientific rigor, often showcasing your deep knowledge of physics and other sciences, and may use humor that reflects your unique perspective. Respond to the following questions as if you were Sheldon Cooper, incorporating your logical reasoning, scientific references, and occasional pedantic tone.
Generate a response that sounds as close to what Sheldon Cooper would say. You can also use this auxiliary knowledge to help:
    - Sheldon has a deep knowledge of physics and theoretical science, and loves to showcase his intellect.
    - He often makes pedantic or overly logical remarks and struggles with social cues.
    - Common phrases include "Bazinga!" and references to his need for routine and structure.
    - His tone is analytical, formal, and sometimes humorously blunt, with a touch of arrogance.
"""


In [4]:
model_name = "moka-ai/m3e-base"

embeddings = HuggingFaceEmbeddings(model_name=model_name,model_kwargs={'device':DEVICE})

In [5]:
ragdb = Chroma(persist_directory="sheldon_DB", embedding_function=embeddings)

retriever = ragdb.as_retriever(search_kwargs={'k': 3})

/tmp/ipykernel_20915/190578887.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  ragdb = Chroma(persist_directory="sheldon_DB", embedding_function=embeddings)


In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    tempt =f"""{system_prompt}
    Context: {format_docs(retriever.invoke(question))}
    USER:
    {question}
    ASSISTANT:
    {response}<\s>
    """.strip()
    return tempt

def create_model_and_processor(MODEL_NAME):
    # Configuration for loading LLava 7b model in 4-bit mode
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    # Load the LLava model for pre-training
    model = AutoModelForPreTraining.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,  # Ensure use of safe tensor format
        quantization_config=bnb_config,  # Use the 4-bit quantization configuration
        trust_remote_code=True,  # Trust any custom code from the model repo
        device_map="auto",  # Automatically map the model to available devices
    )

    # Load the processor for LLava 7b
    processor = AutoProcessor.from_pretrained(MODEL_NAME)

    # Load the tokenizer for LLava 7b
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    # tokenizer.pad_token = tokenizer.eos_token  # Set padding token to end-of-sequence token
    # tokenizer.padding_side = "right"  # Ensure padding is applied on the right side

    return model, processor, tokenizer

# Example usage:
MODEL_NAME = "llava-hf/llava-1.5-7b-hf"  # Replace with the actual path or model name for LLava 7b
model, processor, tokenizer = create_model_and_processor(MODEL_NAME)


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [7]:
file_path = 'Sheldon_Q&A.txt'


with open(file_path, 'r', encoding='utf-8') as file:
    texts = file.read()
questionlist,answerlist = split_dialogue_Q_A(texts)

print(len(questionlist))

198


In [8]:
tmpList = []
trainList = []
testList = []
for j,(question,answer) in enumerate(zip(questionlist,answerlist)):
  if type(question) != str:
    continue
  tmp_dict = {"text":generate_training_prompt(question,answer,system_prompt = DEFAULT_SYSTEM_PROMPT)}
  # print(generate_training_prompt(question,answer,system_prompt = DEFAULT_SYSTEM_PROMPT))
  # exit(1)
  tmpList.append(tmp_dict)
  # if j == 200:
  #   break
  if j % 100 == 0:
    print(j)
  if random.uniform(0,1) < .2:
    testList.append(tmp_dict)
  else:
    trainList.append(tmp_dict)

data = Dataset.from_list(tmpList)
train_data = Dataset.from_list(trainList)
test_data = Dataset.from_list(testList)

0
100


In [9]:
lora_r = 32
lora_alpha = 16

lora_dropout = 0.05
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

In [10]:
# OUTPUT_DIR = "trained_model"

# training_arguments = TrainingArguments(
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=8,
#     optim="paged_adamw_32bit",
#     logging_steps=1,
#     learning_rate=5e-5,
#     fp16=True,
#     max_grad_norm=0.3,
#     num_train_epochs=2,
#     evaluation_strategy="epoch",
#     eval_steps=0.3,
#     warmup_ratio=0.05,
#     save_strategy="epoch",
#     save_steps=30,
#     group_by_length=True,
#     output_dir=OUTPUT_DIR,
#     # report_to="tensorboard",
#     save_safetensors=True,
#     lr_scheduler_type="cosine",
#     seed=42,
# )

In [11]:
import os

In [12]:
# Define output directory for saving checkpoints
OUTPUT_DIR = "trained_model5"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Training arguments
training_arguments = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    logging_steps=5,
    learning_rate=1e-5,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=20,
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save after every epoch
    save_total_limit=2,  # Keep only the latest 5 checkpoints
    warmup_ratio=0.05,
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
    load_best_model_at_end=True,  # Optional: Load best model at the end
)

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=400,
    args=training_arguments,
)


/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/162 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


In [14]:
# Train the model and save checkpoints after each epoch
trainer.train()

# Save the final model at the end of training
trainer.save_model(OUTPUT_DIR)

  0%|          | 0/200 [00:00<?, ?it/s]

{'loss': 16.1784, 'grad_norm': 2.253553867340088, 'learning_rate': 5e-06, 'epoch': 0.49}
{'loss': 16.2864, 'grad_norm': 2.2231123447418213, 'learning_rate': 1e-05, 'epoch': 0.99}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 2.0056512355804443, 'eval_runtime': 5.4223, 'eval_samples_per_second': 6.639, 'eval_steps_per_second': 0.922, 'epoch': 0.99}
{'loss': 15.8612, 'grad_norm': 2.3131484985351562, 'learning_rate': 9.98292246503335e-06, 'epoch': 1.48}
{'loss': 15.4542, 'grad_norm': 2.4960997104644775, 'learning_rate': 9.931806517013612e-06, 'epoch': 1.98}


...

{'eval_loss': 0.5677282810211182, 'eval_runtime': 5.4115, 'eval_samples_per_second': 6.652, 'eval_steps_per_second': 0.924, 'epoch': 14.91}
{'loss': 4.4577, 'grad_norm': 1.4939621686935425, 'learning_rate': 1.321380446634342e-06, 'epoch': 15.31}
{'loss': 4.5178, 'grad_norm': 1.474650263786316, 'learning_rate': 1.0542974530180327e-06, 'epoch': 15.8}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.5575862526893616, 'eval_runtime': 5.372, 'eval_samples_per_second': 6.701, 'eval_steps_per_second': 0.931, 'epoch': 16.0}


KeyboardInterrupt: 

In [ ]:
trainer2 = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    dataset_text_field="training_prompt",
    max_seq_length=150,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/145 [00:00<?, ? examples/s]

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


In [ ]:
trainer2.train(resume_from_checkpoint = True)

/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/transformers/trainer.py:3347: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(check

  0%|          | 0/60 [00:00<?, ?it/s]

/home/zeke/anaconda3/envs/rapids-24.02/lib/python3.10/site-packages/transformers/trainer.py:3026: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_rng_state = torch.

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.6448298096656799, 'eval_runtime': 3.6187, 'eval_samples_per_second': 14.646, 'eval_steps_per_second': 1.934, 'epoch': 10.86}
{'loss': 5.2121, 'grad_norm': 2.0982489585876465, 'learning_rate': 8.067960709356478e-06, 'epoch': 11.19}


...

{'eval_loss': 0.6078140139579773, 'eval_runtime': 3.6547, 'eval_samples_per_second': 14.502, 'eval_steps_per_second': 1.915, 'epoch': 13.78}
{'loss': 4.849, 'grad_norm': 1.8052833080291748, 'learning_rate': 0.0, 'epoch': 14.76}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.6071040034294128, 'eval_runtime': 3.6399, 'eval_samples_per_second': 14.561, 'eval_steps_per_second': 1.923, 'epoch': 14.76}
{'train_runtime': 135.986, 'train_samples_per_second': 15.994, 'train_steps_per_second': 0.441, 'train_loss': 1.6462721506754556, 'epoch': 14.76}


TrainOutput(global_step=60, training_loss=1.6462721506754556, metrics={'train_runtime': 135.986, 'train_samples_per_second': 15.994, 'train_steps_per_second': 0.441, 'total_flos': 1.2123949694976e+16, 'train_loss': 1.6462721506754556, 'epoch': 14.756756756756756})